In [1]:
# Imports

import pandas as pd
from datetime import datetime
from newsdataapi import NewsDataApiClient

from time import sleep
from tqdm.notebook import tqdm

In [2]:
# Create a variable for which file to mess with
file_name = "Finetuning flooding data 15-01-24"

In [3]:
# Key from https://newsdata.io/api-key
key = "..."

# API key authorization, Initialize the client with your API key
api = NewsDataApiClient(apikey=key)

# main = "rain"
# sub = ['flooding', 'floods', 'storm', 'storms', 'hurricane', 'typhoon', 'drown', 'drowning', 'rip currents', 'waves']
# exclude = ['Elon Musk', 'Israel', 'oil', 'market', 'price', 'purchases']

q = 'rain AND (flooding OR floods OR storm OR storms OR hurricane OR typhoon OR drown OR drowning OR rip currents OR waves) AND NOT (Elon Musk OR Israel OR oil OR market OR price OR purchases)'

# Get news articles about rain
response = api.news_api(q=q)
response["nextPage"]

'1705453046918493708'

In [5]:
data = pd.DataFrame(columns=["Date Added", "Article ID", "Link", "Content"])
excel = pd.read_excel(file_name + ".xlsx")
data = pd.concat([data,excel], ignore_index=True)

# page = "1704372960200380441"
# page = '1703948986542190647' # 31-12-23
# page = '1704971777519892679' # 11-01-24
# page = '1705326151098664889' # 15-01-24

# sleepy_time = 60*1 # Seconds
# batch = 1 # pages

# while True:
#     print(datetime.now())
    
#     for i in range(batch):
    
#         response = api.news_api(q=q, page=page, language='en')
#         page = response["nextPage"]

#         for article in response['results']:

#             if article["article_id"] in list(data["Article ID"]):
#                 continue

#             new_record = pd.DataFrame([{'Date Added':datetime.today().strftime('%d-%m-%Y'), 'Article ID':article["article_id"], 'Link':article['link'], 'Content':article['content']}])
#             data = pd.concat([data, new_record], ignore_index=True)

#         data.to_excel(file_name + ".xlsx", index=False)

#         print("Total articles collected: " + str(len(data)))
#         print("Next page: " + str(page) + "\n")
        
#     sleep(sleepy_time) # Zzz...

In [6]:
data["Content"][11]

'New Zealand captain Mitchell Santner took four wickets as they beat Bangladesh by 17 runs in a rain-affected third and final Twenty20 international on Sunday to draw the series 1-1. After dismissing the tourists for 110 in Mount Maunganui, New Zealand\'s top-order collapsed to 49-5 before Jimmy Neesham and Santner recovered to reach 95-5 off 14.4 overs. When heavy rain halted play, the Black Caps were comfortably ahead of the Duckworth-Lewis-Stern target, thanks largely to an unbeaten 28 from all-rounder Neesham. The result denied Bangladesh a first series victory on New Zealand soil in any format, having lost the preceding ODI series 2-1. "Our bowlers did the job up front keeping them to 110 and it\'s good to get across the line in what\'s been a pretty tough-fought series," Santner said. "It got quite tight in the end today. Credit to Bangladesh, their bowlers have bowled extremely well. They put us under some real pressure again." After asking Bangladesh to bat first on a lively Ba

In [7]:
qfef = 'LONDON (AP) — Eurostar services to and from London were canceled Saturday after a tunnel under the River Thames became flooded, disrupting festive travel plans. Hundreds of travelers trying to get across the English Channel were stranded at London’s St. Pancras International station and the Gare du Nord station in Paris. Eurostar, which runs services from London to Paris, Brussels and Amsterdam, said it canceled at least 29 trains Saturday and is not expected to run any services for the rest of the day because of the flooding. Engineers working on the tunnel said water levels were reducing. The volume of water in the tunnel was “unprecedented," they said. The U.K. has been battered by strong, gusty winds and heavy rain brought by Storm Gerrit throughout the festive period. More stormy weather and travel disruption is expected during the last weekend of the year. Chris Dillashaw, from San Antonio, Texas, was among many whose plans for New Year’s Eve were ruined by the travel chaos. “Our entire family is here. ... We were celebrating Christmas in Paris and then headed to London for our New Year’s Eve plans,” he told The Associated Press while waiting at Gare du Nord. “It’s pretty disappointing to find out via an email what happened.” Christina David, 25, and Georgina Benyamin, 26, from Sydney, said they have nowhere to stay after finding out that their train from London to Paris — their final stop in a weeks-long European tour — was canceled. “We paid for an expensive hotel with an Eiffel Tower view,” Benyamin said. “Now we have to book a hotel to stay for the night here. We don’t know where to go; we have nowhere to stay.” Eurostar said it was “extremely sorry for the unforeseen issues affecting our customers and services.” “We understand this is a vital time to get home at the end of the festive season and ahead of New Year," the company said. Eurostar services were also disrupted just before Christmas due to a strike by staff at Eurotunnel. The U.K.’s weather forecaster, the Met Office, said more high winds and rain are expected to hit London and southern England on Saturday. Gusts of up to 50 miles per hour (80 kph) are expected, with the strongest winds likely near coastal areas. Sylvia Hui, The Associated Press'
len(qfef)

2254

In [8]:
def wrap(s, w=50):
    return [s[i:i + w] for i in range(0, len(s), w)]

wrap(qfef)

['LONDON (AP) — Eurostar services to and from London',
 ' were canceled Saturday after a tunnel under the R',
 'iver Thames became flooded, disrupting festive tra',
 'vel plans. Hundreds of travelers trying to get acr',
 'oss the English Channel were stranded at London’s ',
 'St. Pancras International station and the Gare du ',
 'Nord station in Paris. Eurostar, which runs servic',
 'es from London to Paris, Brussels and Amsterdam, s',
 'aid it canceled at least 29 trains Saturday and is',
 ' not expected to run any services for the rest of ',
 'the day because of the flooding. Engineers working',
 ' on the tunnel said water levels were reducing. Th',
 'e volume of water in the tunnel was “unprecedented',
 '," they said. The U.K. has been battered by strong',
 ', gusty winds and heavy rain brought by Storm Gerr',
 'it throughout the festive period. More stormy weat',
 'her and travel disruption is expected during the l',
 'ast weekend of the year. Chris Dillashaw, from San',
 ' Antonio

In [9]:
# Filter on length, too long (+4000 characters) gets filtered out as wordy spam
# Too short is probably also trash
# Filter out duplicates. Divide in parts and see if those parts are in other articles.

drop_i = []
data_copy = data.copy()

# First filter on length since that's O(n)
for i, d in enumerate(data["Content"]):
    if type(d) is not str:
        drop_i.append(i)
        continue
        
    # A bit arbitrary but this seems to be a good length for news articles about floodings
    if len(d) > 4000 or len(d) < 100:
        drop_i.append(i)
        
data_short = data_copy.drop(drop_i)

# Check if fragment in LATER articles
# If true, add that LATER article to the list of indices to be dropped
drop_i = []
data_filtered = data_short.copy().reset_index(drop=True)

for i, article in tqdm(enumerate(data_short["Content"]), total=len(data_short)):
        
    article_chunks = wrap(article)

    for j, other_article in enumerate(data_short["Content"].iloc[i+1:len(data_short)]):

        j = j+i+1 # Only check LATER articles, update j to reflect that

        for chunk in article_chunks:
            if chunk in other_article and len(chunk) > 30:
                drop_i.append(j)
        
drop_i = list(set(drop_i))

# print(drop_i)

data_filtered = data_filtered.drop(drop_i)
data_filtered = data_filtered.drop_duplicates(subset=['Content'], keep='first') # Just to be sure

print("Number of original articles:", len(data))
print("Number of filtered articles:", len(data_filtered))

data_filtered = data_filtered.reset_index(drop=True)
data_filtered.to_excel(file_name + ".xlsx", index=False)

data_filtered

  0%|          | 0/816 [00:00<?, ?it/s]

Number of original articles: 816
Number of filtered articles: 816


,Date Added,Article ID,Link,Content,English,Tagalog
0,31-12-2023,b00dd13a13f407d58b6cdde1954c5230,https://michaelwest.com.au/aussies-welcome-202...,Australians turned out in their hundreds of th...,Australians turned out in their hundreds of th...,Ang mga Australiano ay naka -out sa kanilang d...
1,31-12-2023,71083264ac6170e2f7276e592f60d6bc,https://en.tempo.co/read/1815646/bmkg-warns-pe...,"TEMPO.CO , Jakarta - The Meteorology, Climatol...",NaN,NaN
2,31-12-2023,a71005a9e24402ad8a694ab38b4fe22d,https://www.princegeorgecitizen.com/environmen...,BERLIN (AP) — German Chancellor Olaf Scholz to...,NaN,NaN
3,31-12-2023,33ad2d92e734af9ab2f7f7119c2d2484,https://www.mirror.co.uk/news/uk-news/new-year...,More than 200 flood warnings and alerts are in...,NaN,NaN
4,31-12-2023,1f37928c612e33121b18aa4f19e3ff20,https://www.countytimes.co.uk/news/24019955.fl...,POWYS will see in 2024 amid the threat if floo...,NaN,NaN
...,...,...,...,...,...,...
811,15-01-2024,7a22bbdd8254e880945f0561dc87dc38,https://abc13.com/houston-weather-winter-freez...,Ahead of an arctic freeze arriving early next ...,NaN,NaN
812,15-01-2024,c66d0009b605e73cda4f469bb2241366,https://kildare-nationalist.ie/2024/01/15/free...,"By Cate McCurry, PA Snow and ice have been cau...",NaN,NaN
813,15-01-2024,d461a6d07b5a2b2f76e2064d1f185129,https://www.birminghammail.co.uk/whats-on/shop...,"Tesco, Morrisons and Sainsbury's have introduc...",NaN,NaN
814,15-01-2024,3e24139325dc0ec84cecd387f2db1194,https://allafrica.com/stories/202401150397.html,Harare — Emergency personnel are working throu...,NaN,NaN


In [10]:
# data_filtered["Content"][66]

In [11]:
# Automatic translation

string = ""

for sentence in data_filtered["Content"][0].split(". "):
    string += sentence + ".\n"
    
print(string)

Australians turned out in their hundreds of thousands to see in the new year under a midnight sky lit up by fireworks – the harbour bridge an iconic backdrop for many.
In Sydney, crowds were abuzz and at maximum capacity as revellers crammed picnic rugs together across harbour foreshore vantage points.
More than 8.5 tonnes of fireworks went up in smoke alongside 80,000 pyrotechnic effects and the sounds of jubilant cheers.
For some, it was a 17-hour wait after queuing from the early hours of the morning to snag a premium spot.
But it was worth it as the clock struck 12 for the families, friends, world travellers and locals alike who enjoyed the front-row display of pyrotechnics, a smoking ceremony, AI-driven projections onto the harbour bridge and illuminated boats on the water.
Smaller groups were seen dancing on the pylons of the Harbour Bridge while children looked on in wonder.
By 5pm most major vantage points around the foreshore had already reached capacity with the Sydney Opera 

In [12]:
def get_translation(article, translator):
    
    eng_str = ""
    tl_str = ""
    
    for sentence in tqdm(article.split('. ')):
        try:
            result = translator.translate(sentence + '.', dest="tl", src="en").text

            tl_str += result + "\n"
            eng_str += sentence + '.\n'
            
            sleep(1)
        except:
            print("Timed out! Sleeping now.")
            print(datetime.now())
            sleep(120)
            return np.nan, np.nan
        
    return tl_str, eng_str

In [16]:
# Translate by sentence, before merging back together

from pybacktrans import BackTranslator
from googletrans import Translator
from tqdm.notebook import tqdm
from time import sleep
import numpy as np

translator = Translator()

data = pd.DataFrame(columns=["Date Added", "Article ID", "Link", "Content"])
excel = pd.read_excel(file_name + ".xlsx")
data = pd.concat([data,excel], ignore_index=True)
# data["English"] = [np.nan for x in data["Content"]]
# data["Tagalog"] = [np.nan for x in data["Content"]]

data_copy = data.copy()

# Just to prevent a possible error at the end
data_copy["English"] = ["" for x in data_copy["Content"]]
data_copy["Tagalog"] = ["" for x in data_copy["Content"]]

english = []
tagalog = []

for i, article in tqdm(enumerate(data_copy["Content"]), total=len(data_copy)):
    
#     print(i)
    
    # Check if result already exists
    if type(data["Tagalog"][i]) is str:
        english.append(data["English"][i])
        tagalog.append(data["Tagalog"][i])
        data_copy["English"][i] = english[i]
        data_copy["Tagalog"][i] = tagalog[i]
        data_copy.to_excel(file_name + ".xlsx", index=False)
        continue
    
    # Check if article is not empty
    if type(article) is not str:
        english.append("No article found")
        tagalog.append("No article found")
        data_copy["English"][i] = "No article found"
        data_copy["Tagalog"][i] = "No article found"
        data_copy.to_excel(file_name + ".xlsx", index=False)
        continue
            
    tl_str, eng_str = get_translation(article, translator)
    
    english.append(eng_str)
    tagalog.append(tl_str)
    
    data_copy["English"][i] = english[i]
    data_copy["Tagalog"][i] = tagalog[i]
    data_copy.to_excel(file_name + ".xlsx", index=False)
    
    sleep(30) # To not overwhelm Google Translate
    
data_copy["English"] = english
data_copy["Tagalog"] = tagalog
data_copy.to_excel(file_name + " complete.xlsx", index=False)

data_copy

# Note: manually duplicate the file before interrupting this process!
# (If you time the interrupting wrong you corrupt the entire file and you can start over. Not recommended.)

  0%|          | 0/816 [00:00<?, ?it/s]

,Date Added,Article ID,Link,Content,English,Tagalog
0,31-12-2023,b00dd13a13f407d58b6cdde1954c5230,https://michaelwest.com.au/aussies-welcome-202...,Australians turned out in their hundreds of th...,Australians turned out in their hundreds of th...,Ang mga Australiano ay naka -out sa kanilang d...
1,31-12-2023,71083264ac6170e2f7276e592f60d6bc,https://en.tempo.co/read/1815646/bmkg-warns-pe...,"TEMPO.CO , Jakarta - The Meteorology, Climatol...","TEMPO.CO , Jakarta - The Meteorology, Climatol...","TEMPO.CO, JAKARTA - Ang meteorology, climatolo..."
2,31-12-2023,a71005a9e24402ad8a694ab38b4fe22d,https://www.princegeorgecitizen.com/environmen...,BERLIN (AP) — German Chancellor Olaf Scholz to...,BERLIN (AP) — German Chancellor Olaf Scholz to...,BERLIN (AP) - Kumuha ng isang helicopter fligh...
3,31-12-2023,33ad2d92e734af9ab2f7f7119c2d2484,https://www.mirror.co.uk/news/uk-news/new-year...,More than 200 flood warnings and alerts are in...,More than 200 flood warnings and alerts are in...,Mahigit sa 200 mga babala sa baha at mga alert...
4,31-12-2023,1f37928c612e33121b18aa4f19e3ff20,https://www.countytimes.co.uk/news/24019955.fl...,POWYS will see in 2024 amid the threat if floo...,POWYS will see in 2024 amid the threat if floo...,Makikita sa Powys sa 2024 sa gitna ng banta ku...
...,...,...,...,...,...,...
811,15-01-2024,7a22bbdd8254e880945f0561dc87dc38,https://abc13.com/houston-weather-winter-freez...,Ahead of an arctic freeze arriving early next ...,Ahead of an arctic freeze arriving early next ...,Sa unahan ng isang Arctic freeze na dumating n...
812,15-01-2024,c66d0009b605e73cda4f469bb2241366,https://kildare-nationalist.ie/2024/01/15/free...,"By Cate McCurry, PA Snow and ice have been cau...","By Cate McCurry, PA Snow and ice have been cau...","Sa pamamagitan ng Cate McCurry, ang PA Snow at..."
813,15-01-2024,d461a6d07b5a2b2f76e2064d1f185129,https://www.birminghammail.co.uk/whats-on/shop...,"Tesco, Morrisons and Sainsbury's have introduc...","Tesco, Morrisons and Sainsbury's have introduc...","Ang Tesco, Morrisons at Sainsbury's ay nagpaki..."
814,15-01-2024,3e24139325dc0ec84cecd387f2db1194,https://allafrica.com/stories/202401150397.html,Harare — Emergency personnel are working throu...,Harare — Emergency personnel are working throu...,HARARE-Ang mga tauhan ng emerhensiya ay nagtat...


In [30]:
# A final preprocess, remove entries with a single newline in them

remove_i = []

for i, article in enumerate(data_copy["English"]):
    if len(article.split('\n')) < 3 or "{" in article:
        remove_i.append(i)
        
data_copy.drop(remove_i)

,Date Added,Article ID,Link,Content,English,Tagalog
0,31-12-2023,b00dd13a13f407d58b6cdde1954c5230,https://michaelwest.com.au/aussies-welcome-202...,Australians turned out in their hundreds of th...,Australians turned out in their hundreds of th...,Ang mga Australiano ay naka -out sa kanilang d...
1,31-12-2023,71083264ac6170e2f7276e592f60d6bc,https://en.tempo.co/read/1815646/bmkg-warns-pe...,"TEMPO.CO , Jakarta - The Meteorology, Climatol...","TEMPO.CO , Jakarta - The Meteorology, Climatol...","TEMPO.CO, JAKARTA - Ang meteorology, climatolo..."
2,31-12-2023,a71005a9e24402ad8a694ab38b4fe22d,https://www.princegeorgecitizen.com/environmen...,BERLIN (AP) — German Chancellor Olaf Scholz to...,BERLIN (AP) — German Chancellor Olaf Scholz to...,BERLIN (AP) - Kumuha ng isang helicopter fligh...
3,31-12-2023,33ad2d92e734af9ab2f7f7119c2d2484,https://www.mirror.co.uk/news/uk-news/new-year...,More than 200 flood warnings and alerts are in...,More than 200 flood warnings and alerts are in...,Mahigit sa 200 mga babala sa baha at mga alert...
4,31-12-2023,1f37928c612e33121b18aa4f19e3ff20,https://www.countytimes.co.uk/news/24019955.fl...,POWYS will see in 2024 amid the threat if floo...,POWYS will see in 2024 amid the threat if floo...,Makikita sa Powys sa 2024 sa gitna ng banta ku...
...,...,...,...,...,...,...
811,15-01-2024,7a22bbdd8254e880945f0561dc87dc38,https://abc13.com/houston-weather-winter-freez...,Ahead of an arctic freeze arriving early next ...,Ahead of an arctic freeze arriving early next ...,Sa unahan ng isang Arctic freeze na dumating n...
812,15-01-2024,c66d0009b605e73cda4f469bb2241366,https://kildare-nationalist.ie/2024/01/15/free...,"By Cate McCurry, PA Snow and ice have been cau...","By Cate McCurry, PA Snow and ice have been cau...","Sa pamamagitan ng Cate McCurry, ang PA Snow at..."
813,15-01-2024,d461a6d07b5a2b2f76e2064d1f185129,https://www.birminghammail.co.uk/whats-on/shop...,"Tesco, Morrisons and Sainsbury's have introduc...","Tesco, Morrisons and Sainsbury's have introduc...","Ang Tesco, Morrisons at Sainsbury's ay nagpaki..."
814,15-01-2024,3e24139325dc0ec84cecd387f2db1194,https://allafrica.com/stories/202401150397.html,Harare — Emergency personnel are working throu...,Harare — Emergency personnel are working throu...,HARARE-Ang mga tauhan ng emerhensiya ay nagtat...


In [32]:
# Do a wordcount of total amount of data collected, so you can compare with the eval set

tagalog = data_copy['Tagalog']
total = 0
for sentence in tagalog:
    total += len(sentence.replace('\n',' ').split())
    
print("Total word count in Tagalog: " + str(total))

# Eval set: 14539 words
# Train set: 359688 words

Total word count in Tagalog: 359688


In [33]:
# Same format as the other data, two separate txt files

np.savetxt(file_name + " EN.txt", data_copy["English"].values, fmt="%s")
np.savetxt(file_name + " TL.txt", data_copy["Tagalog"].values, fmt="%s")